# Introduction to Linear Programming (Solutions)

#### Warmup

Using Gurobi, solve the following problem:  
    max $\; x+y+z$  
    s.t.  
    $x+y \leq 10$  
    $y + 3z \leq 2x$  
    $3x + 2y + z \leq 20$  
    $x, y, z \geq 0$  

In [4]:
import gurobipy as grb

env = grb.Env.ClientEnv("gurobi.log",
                        "10.116.90.112")
model = grb.Model("LP_Warmup", env)

x = model.addVars(3, lb=0)

model.setObjective(x[0] + x[1] + x[2], grb.GRB.MAXIMIZE)

constr1 = model.addConstr(x[0] + x[1] <= 10)
constr2 = model.addConstr(x[1] + 3*x[2] <= 2*x[0])
constr3 = model.addConstr(3*x[0] + 2*x[1] + x[2] <= 20)

model.optimize()

print(x)

del model
del env

Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (win64)
Optimize a model with 3 rows, 3 columns and 8 nonzeros
Model fingerprint: 0x29e5e92d
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+01, 2e+01]
Presolve time: 0.00s
Presolved: 3 rows, 3 columns, 8 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    3.0000000e+30   4.000000e+30   3.000000e+00      0s
       4    9.0909091e+00   0.000000e+00   0.000000e+00      0s

Solved in 4 iterations and 0.00 seconds
Optimal objective  9.090909091e+00
{0: <gurobi.Var C0 (value 5.454545454545455)>, 1: <gurobi.Var C1 (value 0.0)>, 2: <gurobi.Var C2 (value 3.6363636363636367)>}

Compute Server communication statistics:
  Sent: 0.003 MBytes in 20 msgs and 1.95s (0.00 MB/s)
  Received: 0.002 MBytes in 6 msgs and 0.56s (0.00 MB/s)



#### RunDisney Example

In [5]:
import gurobipy as grb
import pandas as pd

# Input Data 
# Product Information
prod_df = pd.DataFrame({
    'Product': ['5K', '10K', 'Half', 'Marathon', 'Goofy', 'Dopey'],
    'Price': [80, 120, 180, 190, 365, 560],
    'Demand': [15692, 11896, 18175, 17604, 4325, 5349],
    'Min_Spots': [10000, 7000, 5000, 5000, 2000, 2000],
})

prod_df.set_index('Product', inplace=True)
prod_list = prod_df.index.values

# Resource Information
res_df = pd.DataFrame({
    'Resource': ['5K', '10K', 'Half', 'Marathon'],
    'Capacity': [15000, 15000, 25000, 25000],
    'Products': [['5K', 'Dopey'], ['10K', 'Dopey'], 
                 ['Half', 'Goofy', 'Dopey'], ['Marathon', 'Goofy', 'Dopey']],
})

res_df.set_index('Resource', inplace=True)
res_list = res_df.index.values

# Connect to environment and build model
env = grb.Env.ClientEnv("gurobi.log",
                        "10.116.90.112")
model = grb.Model("Marathon", env)

# Create a decision variable: the number of spots to make available for registration for each of the six products.
spots = model.addVars(prod_list, lb=0)

# Build the objective
exp_rev = grb.quicksum( 
    spots[prod]*prod_df['Price'][prod] for prod in prod_list )

total_exp_rev = model.setObjective(exp_rev, grb.GRB.MAXIMIZE)

# Allow no more spots than expected demand. 
# For purposes here, we assume opening any more spots will not lead to any more bookings
demand_constraint = model.addConstrs((spots[prod] <= prod_df['Demand'][prod] for prod in prod_list), 
                                     'demand_constraint')

# Adhere to the minimum value constraints.
min_spot_constraint = model.addConstrs((spots[prod] >= prod_df['Min_Spots'][prod] for prod in prod_list), 
                                       'min_spot_constraint')

# Capacity constraints for each resource
capacity_constraint = model.addConstrs((grb.quicksum(spots[prod] for prod in res_df['Products'][res]) 
                                        <= res_df['Capacity'][res] for res in res_list), 
                                       'capacity_constraint')

model.optimize()

# Generate output as a data frame
spots_values = model.getAttr('x',spots)
prod_df['opt_spots'] = pd.Series(spots_values)

print(prod_df)

del model
del env

Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (win64)
Optimize a model with 16 rows, 6 columns and 22 nonzeros
Model fingerprint: 0x92b19ae0
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [8e+01, 6e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+03, 2e+04]
Presolve removed 13 rows and 1 columns
Presolve time: 0.00s
Presolved: 3 rows, 5 columns, 8 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.4074885e+07   1.678625e+03   0.000000e+00      0s
       2    1.2197500e+07   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.00 seconds
Optimal objective  1.219750000e+07
          Price  Demand  Min_Spots  opt_spots
Product                                      
5K           80   15692      10000    11896.0
10K         120   11896       7000    11896.0
Half        180   18175       5000    17604.0
Marathon    190   17604       5000    17604.0
Goofy       365    4325       2000     4292.0
Dopey 